In [1]:
# pip install numpy pandas scikit-learn tensorflow matplotlib

## Tratamento dos Dados

In [4]:
import pandas as pd

# Carregar os dados
file1_path = '/media/bishopx/Dados/MESTRADO/RP/rec_pad/House price data - 2016.csv'
file2_path = '/media/bishopx/Dados/MESTRADO/RP/rec_pad/kc_house_data.csv'

data1 = pd.read_csv(file1_path)
data2 = pd.read_csv(file2_path)


In [13]:
data1.rename(columns={
    'number of bedrooms': 'bedrooms',
    'number of bathrooms': 'bathrooms',
    'living area': 'sqft_living',
    'lot area': 'sqft_lot',
    'number of floors': 'floors',
    'waterfront present': 'waterfront',
    'number of views': 'view',
    'condition of the house': 'condition',
    'grade of the house': 'grade',
    'Area of the house(excluding basement)': 'sqft_above',
    'Area of the basement': 'sqft_basement',
    'Built Year': 'yr_built',
    'Renovation Year': 'yr_renovated',
    'Lattitude': 'lat',
    'Longitude': 'long',
    'Postal Code': 'zipcode',
    'Price': 'price'
}, inplace=True)

In [18]:
data1.dtypes

id                             int64
Date                          object
bedrooms                       int64
bathrooms                    float64
sqft_living                    int64
sqft_lot                       int64
floors                       float64
waterfront                     int64
view                           int64
condition                      int64
grade                          int64
sqft_above                     int64
sqft_basement                  int64
yr_built                       int64
yr_renovated                   int64
zipcode                        int64
lat                          float64
long                         float64
living_area_renov              int64
lot_area_renov                 int64
Number of schools nearby       int64
Distance from the airport      int64
price                          int64
dtype: object

In [19]:
# Pré-processamento da primeira base de dados
data1['bathrooms'] = data1['bathrooms'].astype(str).str.replace(',', '.').astype(float)
data1['floors'] = data1['floors'].astype(str).str.replace(',', '.').astype(float)
data1['lat'] = data1['lat'].astype(str).str.replace(',', '.').astype(float)
data1['long'] = data1['long'].astype(str).str.replace(',', '.').astype(float)



In [21]:
# Identificar colunas comuns entre as duas bases
common_columns = list(set(data1.columns).intersection(set(data2.columns)))

# Filtrar as colunas comuns nas duas bases
data1_filtered = data1[common_columns]
data2_filtered = data2[common_columns]

# Unir as bases
combined_data = pd.concat([data1_filtered, data2_filtered], ignore_index=True)

# Confirmar o resultado
combined_data.head()

,lat,bedrooms,yr_renovated,floors,condition,sqft_lot,sqft_basement,id,bathrooms,waterfront,long,sqft_above,price,grade,view,yr_built,zipcode,sqft_living
0,52.8645,5,0,2.0,5,9050,280,6762810145,2.50,0,-114.557,3370,2380000.0,10,4,1921,122003,3650
1,52.8878,4,0,1.5,5,4000,1010,6762810635,2.50,0,-114.470,1910,1400000.0,8,0,1909,122004,2920
2,52.8852,5,0,1.5,3,9480,0,6762810998,2.75,0,-114.468,2910,1200000.0,8,0,1939,122004,2910
3,52.9532,4,0,2.0,3,42998,0,6762812605,2.50,0,-114.321,3310,838000.0,9,0,2001,122005,3310
4,52.9047,3,0,1.5,4,4500,830,6762812919,2.00,0,-114.485,1880,805000.0,8,0,1929,122006,2710


#### 2. Divisão dos Dados

In [22]:
from sklearn.model_selection import train_test_split

# Separar variáveis independentes (X) e dependentes (y)
X = combined_data.drop(columns=['price', 'id'])
y = combined_data['price']

# Dividir os dados em treino (70%), validação (15%) e teste (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

#### 3. Treinamento de Modelos Base

### SVR

In [23]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Pipeline de pré-processamento e modelo SVR
svr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR(kernel='rbf'))
])

# Treinamento
svr_pipeline.fit(X_train, y_train)

# Avaliação no conjunto de validação
y_pred_val_svr = svr_pipeline.predict(X_val)
svr_rmse = mean_squared_error(y_val, y_pred_val_svr, squared=False)
print(f"RMSE do SVR no conjunto de validação: {svr_rmse}")

RMSE do SVR no conjunto de validação: 382020.4860333773


/home/bishopx/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


#### CNN (Rede Neural Convolucional)

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten
from tensorflow.keras.optimizers import Adam

# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Remodelar os dados para a CNN
X_train_cnn = X_train_scaled.reshape(-1, X_train_scaled.shape[1], 1)
X_val_cnn = X_val_scaled.reshape(-1, X_val_scaled.shape[1], 1)

# Definir o modelo CNN
cnn_model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)  # Saída única para regressão
])

# Compilação e treinamento
cnn_model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])
cnn_model.fit(X_train_cnn, y_train, validation_data=(X_val_cnn, y_val), epochs=10, batch_size=32)

# Avaliação no conjunto de validação
cnn_loss, cnn_mae = cnn_model.evaluate(X_val_cnn, y_val)
print(f"MAE da CNN no conjunto de validação: {cnn_mae}")

2024-11-20 20:32:18.654323: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-20 20:32:18.899380: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-20 20:32:19.101652: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732145539.293469    8212 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732145539.347892    8212 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-20 20:32:19.797327: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Epoch 1/10


/home/bishopx/.local/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-20 20:32:24.688510: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


793/793 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 430543568896.0000 - mae: 540905.9375 - val_loss: 374378463232.0000 - val_mae: 490641.5938
Epoch 2/10
793/793 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 332585697280.0000 - mae: 453401.0625 - val_loss: 176388587520.0000 - val_mae: 274238.4688
Epoch 3/10
793/793 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 139884101632.0000 - mae: 235028.6719 - val_loss: 81674633216.0000 - val_mae: 173742.7656
Epoch 4/10
793/793 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 70063120384.0000 - mae: 170729.2188 - val_loss: 73125117952.0000 - val_mae: 172671.8281
Epoch 5/10
793/793 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 74433208320.0000 - mae: 173283.3281 - val_loss: 69891727360.0000 - val_mae: 168413.4375
Epoch 6/10
793/793 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 65224376320.0000 - mae: 166620.6250 - val_loss: 67091804160.0000 - val_mae: 164794.0625
Epoch 7/10
793/793 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 64515969024.0000 - mae: 163229.1406 - val_loss: 6455447142

#### LSTM (Long short-term memory) | RNN (Recurrent Neural Networking)

In [25]:
from tensorflow.keras.layers import LSTM

# Remodelar os dados para LSTM
X_train_lstm = X_train_scaled.reshape(-1, 1, X_train_scaled.shape[1])
X_val_lstm = X_val_scaled.reshape(-1, 1, X_val_scaled.shape[1])

# Definir o modelo LSTM
lstm_model = Sequential([
    LSTM(50, activation='relu', input_shape=(1, X_train_scaled.shape[1])),
    Dense(1)  # Saída única para regressão
])

# Compilação e treinamento
lstm_model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])
lstm_model.fit(X_train_lstm, y_train, validation_data=(X_val_lstm, y_val), epochs=10, batch_size=32)

# Avaliação no conjunto de validação
lstm_loss, lstm_mae = lstm_model.evaluate(X_val_lstm, y_val)
print(f"MAE do LSTM no conjunto de validação: {lstm_mae}")

/home/bishopx/.local/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
793/793 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 426902749184.0000 - mae: 539846.4375 - val_loss: 425286369280.0000 - val_mae: 534911.1250
Epoch 2/10
793/793 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 440226578432.0000 - mae: 545785.5625 - val_loss: 424729346048.0000 - val_mae: 534526.0625
Epoch 3/10
793/793 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 421082562560.0000 - mae: 539297.6250 - val_loss: 423908081664.0000 - val_mae: 533961.3125
Epoch 4/10
793/793 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 423730151424.0000 - mae: 538041.5000 - val_loss: 422866780160.0000 - val_mae: 533246.7500
Epoch 5/10
793/793 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 427493130240.0000 - mae: 535968.7500 - val_loss: 421635457024.0000 - val_mae: 532402.3125
Epoch 6/10
793/793 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 410706345984.0000 - mae: 533818.8750 - val_loss: 420201070592.0000 - val_mae: 531423.8125
Epoch 7/10
793/793 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 424973434880.0000 - mae: 537048.0625 - v

#### 4. Sistema Híbrido com PSO

PSO is a metaheuristic optimization algorithm that can be used to find the best weights for combining the models. Here's how to implement it:

In [26]:
from scipy.optimize import differential_evolution

# Função objetivo para otimizar pesos dos modelos
def objective(weights):
    final_pred = (weights[0] * y_pred_val_svr +
                  weights[1] * cnn_model.predict(X_val_cnn).flatten() +
                  weights[2] * lstm_model.predict(X_val_lstm).flatten())
    return mean_squared_error(y_val, final_pred, squared=False)

# Restrições para os pesos (0 a 1)
bounds = [(0, 1), (0, 1), (0, 1)]

# Otimização com PSO
result = differential_evolution(objective, bounds, strategy='best1bin', maxiter=100, seed=42)
best_weights = result.x
print(f"Pesos otimizados: {best_weights}")

170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
 54/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

/home/bishopx/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


170/170 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
 30/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

/home/bishopx/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
 50/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

...

/home/bishopx/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
 69/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

/home/bishopx/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Pesos otimizados: [0.         0.98937056 0.27869501]


/home/bishopx/.local/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


## Calculando Metricas

##### 1. Importar as Bibliotecas Necessárias

In [27]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

##### 2. Funções para Calcular as Métricas

In [28]:
# Função para calcular MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Função para calcular MBE
def mean_bias_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(y_pred - y_true)


##### 3. Cálculo de Métricas no Conjunto de Validação

In [29]:
# Previsões (exemplo para um modelo qualquer)
y_pred_val = svr_pipeline.predict(X_val)  # Substitua pelo modelo correspondente

# Cálculo das métricas
mae = mean_absolute_error(y_val, y_pred_val)
mse = mean_squared_error(y_val, y_pred_val)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred_val)
mape = mean_absolute_percentage_error(y_val, y_pred_val)
mbe = mean_bias_error(y_val, y_pred_val)

# Exibir os resultados
print(f"MAE (Erro Médio Absoluto): {mae}")
print(f"MSE (Erro Quadrático Médio): {mse}")
print(f"RMSE (Raiz do Erro Quadrático Médio): {rmse}")
print(f"R² (Coeficiente de Determinação): {r2}")
print(f"MAPE (Erro Percentual Médio Absoluto): {mape}%")
print(f"MBE (Erro Médio de Tendência): {mbe}")

MAE (Erro Médio Absoluto): 218729.201726333
MSE (Erro Quadrático Médio): 145939651749.1778
RMSE (Raiz do Erro Quadrático Médio): 382020.4860333773
R² (Coeficiente de Determinação): -0.04835103945633401
MAPE (Erro Percentual Médio Absoluto): 42.391188994561034%
MBE (Erro Médio de Tendência): -84517.69961043455


As métricas de avaliação dependem do objetivo do modelo, que neste caso é prever o **preço de imóveis**. Para problemas de regressão, como este, as métricas mais relevantes são as que avaliam a proximidade entre os valores preditos pelo modelo e os valores reais. Aqui estão as principais métricas que você pode usar:

---

### **1. Erro Médio Absoluto (MAE - Mean Absolute Error)**
**Fórmula:**
\[
\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |\hat{y}_i - y_i|
\]
- **Interpretação:** Média das diferenças absolutas entre os valores preditos (\(\hat{y}_i\)) e reais (\(y_i\)).
- **Vantagens:** Fácil de interpretar, pois está na mesma unidade da variável alvo (preço).
- **Quando usar:** Ideal quando penalizar todos os erros de forma uniforme é desejável.

---

### **2. Erro Quadrático Médio (MSE - Mean Squared Error)**
**Fórmula:**
\[
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (\hat{y}_i - y_i)^2
\]
- **Interpretação:** Média dos quadrados dos erros. Penaliza mais os erros maiores.
- **Vantagens:** Útil quando erros grandes devem ser penalizados mais severamente.
- **Desvantagens:** Não está na mesma unidade do preço (é o quadrado).
- **Quando usar:** Para modelos onde minimizar grandes desvios é prioritário.

---

### **3. Raiz do Erro Quadrático Médio (RMSE - Root Mean Squared Error)**
**Fórmula:**
\[
\text{RMSE} = \sqrt{\text{MSE}}
\]
- **Interpretação:** Raiz quadrada do MSE. Representa os erros na mesma unidade do preço.
- **Vantagens:** Mais interpretável do que o MSE.
- **Quando usar:** Quando você deseja penalizar mais os erros grandes, mas ainda quer uma métrica na mesma unidade.

---

### **4. Coeficiente de Determinação (R² - R-Squared)**
**Fórmula:**
\[
R^2 = 1 - \frac{\text{SSE}}{\text{SST}}
\]
Onde:
- \(\text{SSE}\): Soma dos erros quadrados do modelo.
- \(\text{SST}\): Soma dos erros quadrados em relação à média dos dados.

- **Interpretação:** Mede o quanto da variação nos dados é explicada pelo modelo (varia de 0 a 1).
- **Vantagens:** Ajuda a entender a qualidade geral do ajuste do modelo.
- **Quando usar:** Para comparar modelos diferentes e entender a proporção de variância explicada.

---

### **5. Mean Absolute Percentage Error (MAPE)**
**Fórmula:**
\[
\text{MAPE} = \frac{100}{n} \sum_{i=1}^{n} \left| \frac{\hat{y}_i - y_i}{y_i} \right|
\]
- **Interpretação:** Percentual médio de erro absoluto.
- **Vantagens:** Fácil de interpretar como percentual.
- **Desvantagens:** Pode ser sensível a valores reais próximos de zero.
- **Quando usar:** Quando a magnitude relativa dos erros é importante.

---

### **6. Mean Bias Error (MBE)**
**Fórmula:**
\[
\text{MBE} = \frac{1}{n} \sum_{i=1}^{n} (\hat{y}_i - y_i)
\]
- **Interpretação:** Mede a tendência média do modelo (superestimativa ou subestimativa).
- **Quando usar:** Para avaliar se o modelo está tendendo a superestimar ou subestimar.

---

### Resumo das Métricas a Usar
Para a avaliação do seu modelo:
1. **MAE:** Para erros absolutos e interpretação direta.
2. **RMSE:** Para penalizar erros grandes e manter a unidade.
3. **R²:** Para avaliar a capacidade explicativa geral.
4. **MAPE:** Para uma perspectiva percentual, especialmente em casos com preços variando em ordens de magnitude.

**Recomendação:** Use uma combinação de pelo menos **MAE, RMSE e R²** para obter uma avaliação abrangente. Se precisar de ajuda para implementá-las, posso fornecer o código.

#### 2. Optimize Weights with Particle Swarm Optimization (PSO)


In [ ]:
from scipy.optimize import differential_evolution
import numpy as np
from sklearn.metrics import mean_squared_error

# Example predictions from models (replace with your models' predictions)
y_pred_svr = svr_pipeline.predict(X_val)
y_pred_cnn = cnn_model.predict(X_val_cnn).flatten()
y_pred_lstm = lstm_model.predict(X_val_lstm).flatten()

# Objective function for PSO (to minimize RMSE)
def objective(weights):
    weights = np.array(weights)
    final_pred = (
        weights[0] * y_pred_svr +
        weights[1] * y_pred_cnn +
        weights[2] * y_pred_lstm
    )
    return mean_squared_error(y_val, final_pred, squared=False)

# Constraints for weights (sum to 1)
bounds = [(0, 1), (0, 1), (0, 1)]

# Differential evolution to optimize weights
result = differential_evolution(objective, bounds, strategy='best1bin', maxiter=100, seed=42)
optimal_weights = result.x
print(f"Optimal Weights: {optimal_weights}")

# Final predictions using optimized weights
final_predictions = (
    optimal_weights[0] * y_pred_svr +
    optimal_weights[1] * y_pred_cnn +
    optimal_weights[2] * y_pred_lstm
)

# Evaluate final model
rmse = mean_squared_error(y_val, final_predictions, squared=False)
print(f"Optimized RMSE: {rmse}")